In [73]:
### Adam Bruce, Started December, 2024. 
### Data was current as of January 1st, 2025
### The following code/descriptors help solve the assignment optimization problem for Turf Badger's Stevens Point 
# Office (S Corporation)

### Load Libraries
import numpy as np
import pandas as pd
new_space = lambda: print(" ")

### Load the dataset

# The dataset customers.csv contains information on ALL active, recurring customer accounts!
# Cleaning will need to be performed based on information found in the data

SP_customers = pd.read_csv('Customers.csv')

# If wanting A quick look at the dataset:
#print(SP_customers.head(2))
#new_space()

# Count of Customers in the dataset:
print(f'The number of Active customers is: {SP_customers.shape[0]}')


The number of Active customers is: 2311


In [75]:
### Cleaning 
### NOTE: Some # values by print statements are used to reduce clutter, to look at the output undue the # by the print statement!
### NOTE: Because the latitude and longitude are available, we do not need the address column for location purposes.
# It could be useful for the administrator, Tricia, later on though, so it is kept.


## To start, column names will be underscored and condensed to make work going forward easier
# For example, Supscription Status will become Subscription_Status
SP_customers.rename(columns={'Customer ID': 'Cust_ID', 'Last Name': 'Last', 'First Name': 'First', 'Sold Date': 'Sold_Date', 'Company Name': 'Company_Name',
                            'Subscription Contract Value': 'Contract_Value', 'Subscription Category': 'Subscription_Category'}, inplace=True)

# Now, nan values must be addressed in the dataset.
# First, lets grab the company customers from the Company_Name column in the dataset
company_data = SP_customers.loc[~SP_customers['Company_Name'].isna(), 'Company_Name']
print(company_data)
new_space()
# There are three companies found here:Hilton Garden Inn, Bridge Street Partners, LLP, and Lazy Meadows Homes LLC

# Now, lets look at the NA rows for the FIRST and LAST NAME Columns
First_nan_data = SP_customers[SP_customers['First'].isna()]
# print(First_nan_data)
# new_space()
# There are no cases where First Name is Empty! Thus, we can look to see if the first names match the company names
# if this is true we can drop the company name column We will filter by indexes found above

First_company_data = SP_customers.filter(items=[1008, 1862, 2242], axis=0)
# print(First_company_data['First'])
# new_space()

# We see the owners are listed as first and last names in this case. We will replace the owners with the company name
# Last will be replaced with a blank string " "

# Make the company name dataframe (undue # note to print)
company_data = pd.DataFrame(company_data)
# print(company_data)

# Replace first with company name in cases of interest
SP_customers.loc[company_data.index, 'First'] = company_data['Company_Name']
# Now replace the index values for LAST with blank strings!
SP_customers.loc[1005,'Last'] = ""
SP_customers.loc[1856,'Last'] = ""
SP_customers.loc[2236,'Last'] = ""

# Now look at the dataframe for these specific indices!
new_company_data = SP_customers.filter(items=[1008, 1862, 2242], axis=0)
#print(new_company_data)
#new_space()

# NOW WE CAN DROP THE COMPANY NAME COLUMN ENTIRELY!
SP_customers_2 = SP_customers.drop(columns=['Company_Name'])

# We now recheck the NA values remaining
null_data_1 = SP_customers_2[SP_customers_2.isnull().any(axis=1)]
# print(null_data_1)
# new_space()

# There is one nan value remaining for row 243, a company that has no last name value. It will be replaced with a blank string like the other companies!
SP_customers_2.loc[243,'Last'] = ""

# Now lets get a count of the null data left!
print(f'There are {SP_customers_2.isnull().sum().sum()} rows with missing data remaining.')
new_space()

# We create the final "Customer" column by combining First and Last! We also drop the combined columns
SP_customers_2['Customer'] = SP_customers_2['First'] + " " + SP_customers_2['Last']
SP_customers_3 = SP_customers_2.drop(columns=['First', 'Last'])

# Check the head of this dataset and for null values (should be none)
# print(SP_customers_3.head(1))
# print(f'There are {SP_customers_3.isnull().sum().sum()} rows with missing data remaining.')



## Next we will address the total yearly services for each contract. 
# Sold_Date indicates the month of service start for each customer. We need only the month from this date column.
# So we will extract it and make it a new column "Start_Month" before dropping the sold_date column

SP_customers_3[['Start_Month','Start_Day', 'Start_Year']] = SP_customers_3['Sold_Date'].str.split('/',expand=True)
#print(SP_customers_3.head(1))
#new_space()

# Drop the unneeded columns
SP_customers_4 = SP_customers_3.drop(['Sold_Date','Start_Day','Start_Year'], axis=1)
#print(SP_customers_4.head(1))
#new_space()

## We also drop the Address column because it contains sensitive information that isn't needed for solving the problem.
## because we can cluster based on latitude and longitude!

SP_customers_4 = SP_customers_4.drop(['Address'], axis=1)


## The service entries X Fleat & Tick and Lawn - FERT TEMP are data entry errors by sales representatives
## The X FLEA & TICK should just be FLEA & TICK while 'Lawn - FERT TEMP' should be 'Fertilization' Per Tricia

# Identify unique subsciption types
#unique_services = SP_customers_4['Subscription'].unique()
#print(unique_services)

# Identify Data Entry Errors and fix
# X Flea & Tick to Flea & Tick & 'Lawn - FERT/TEMP' to Fertilization
SP_customers_4.Subscription = SP_customers_4.Subscription.replace({'X Flea & Tick': "Flea & Tick", 'Lawn - FERT TEMP': "Fertilization"})
new_space() 

## Per the Administrator Tricia and branch owner James, bed bug, 'WL - Foundation Exclusion', 'Flea & Tick',
## 'WL - Wildlife/Full Exclusion Services', and 'WL - Rodent/Wildlife Trapping'
## are single services that need not be accounted for in a service zone total.
## So rows of these service types will be dropped.

SP_customers_4 = SP_customers_4.drop(SP_customers_4[(SP_customers_4['Subscription'] == 'WL - Rodent/Wildlife Trapping') |
                                                   (SP_customers_4['Subscription'] == 'Flea & Tick') |
                                                   (SP_customers_4['Subscription'] == 'WL - Foundation Exclusion') |
                                                   (SP_customers_4['Subscription'] == 'WL - Wildlife/Full Exclusion Services') |
                                                   (SP_customers_4['Subscription'] == 'Pest - Bed Bug')].index)

# Identify if the rows were properly dropped by looking at the unique services!
# Identify unique subsciption types
# unique_services = SP_customers_4['Subscription'].unique()
# print(unique_services)

## An important constraint for the zones will be the net income per year for each technician. 
## Per Tricia, the net per year MUST BE GREATER THAN OR EQUAL TO 20,000 USD
## Per Tricia, the net takehome per service contract total for technicians is 14%
## Thus, a new column, Tech_Takehome will be calculated at 0.14*Contract_Value

# Create the new column
SP_customers_4['Tech_Takehome'] = round(SP_customers_4['Contract_Value'] * 0.14, 2)
#print(SP_customers_4.head(1))
#new_space()

# Count final number of Customers in the dataset:
print(f'The final number of Active customers is: {SP_customers_4.shape[0]}')

## Now we save the cleaned data to a csv file!

SP_customers_4.to_csv('SP_Customers_Clean.csv')


1005              Hilton Garden Inn
1856    Bridge Street Partners, LLP
2236         Lazy Meadows Homes LLC
Name: Company_Name, dtype: object
 
There are 0 rows with missing data remaining.
 
 
The final number of Active customers is: 2282
